In [1]:
!export PYSPARK_PYTHON=python3.6.9
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout,Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess
import tensorflow as tf
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.corpus import brown
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit
import pickle
import tensorflow.keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json
from tensorflow import keras

spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '3'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.cores', '5'),
 ('spark.executor.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.ui.proxyBase', '/proxy/application_1643192276452_0074'),
 ('spark.driver.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.host', 'kafka1'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://0.0.0.0:8089/proxy/application_1643192276452_0074'),
 ('spark.yarn.archive', 'hdfs:///user/spark/conf/spark-libs.jar'),
 ('spark.app.id', 'application_1643192276452_0074'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.kryoserializer.buffer.max', '2047'),
 (

In [2]:
# 임시코드
csi_pos_neg = spark.read.csv("hdfs:///user/spark/datafile/csiposneg.csv", header=True)
csi_pos_neg = csi_pos_neg.withColumn("label",col("label").cast("integer"))

In [3]:
f = open('/root/keywordset.txt', 'r', encoding='utf-8')
keyfile = f.readline()
keyword_list = list()
keywords = keyfile.split(',')
for keyword in keywords:
    keyword_list.append(keyword.strip())

In [4]:
t1 = csi_pos_neg.collect()[:24939]
t2 = csi_pos_neg.collect()[24939:49878]
t3 = csi_pos_neg.collect()[49878:89756]
t4 = csi_pos_neg.collect()[89756:99756]

t5 = csi_pos_neg.collect()[99756:124695]
t6 = csi_pos_neg.collect()[124695:149634]
t7 = csi_pos_neg.collect()[149634:189512]
t8 = csi_pos_neg.collect()[189512:199512]

t1 = spark.createDataFrame(t1)
t2 = spark.createDataFrame(t2)
t3 = spark.createDataFrame(t3)
t4 = spark.createDataFrame(t4)

t5 = spark.createDataFrame(t5)
t6 = spark.createDataFrame(t6)
t7 = spark.createDataFrame(t7)
t8 = spark.createDataFrame(t8)

first_df = t1.union(t5).union(t2).union(t6) #.union(t3).union(t7) # 179512 
first_df_df = t3.union(t7)
second_df = t4.union(t8) # 20000 -> unrevealed data for all periods
first_df = first_df.drop(col('_c0'))
first_df_df = first_df_df.drop(col('_c0'))
second_df = second_df.drop(col('_c0'))

In [5]:
# get_tokens 함수과 같은 용도
stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
# get_lemma 함수와 같은 용도
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

mid_pipeline = Pipeline(stages=[stage_1, stage_2])
mid_preprocessing = mid_pipeline.fit(csi_pos_neg)
mid_preprocessed = mid_preprocessing.transform(csi_pos_neg)

In [6]:
##### Glove embedding #####

In [7]:
class vocabulary:
    staticList = list()
    
class label:
    staticList = list()

In [8]:
from gensim.test.utils import datapath
from gensim.test.utils import get_tmpfile, common_dictionary, common_corpus
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# load the vectors. other vector sizes were used but yielded generally less sensible models
glove_file = datapath("/root/spark/glove.6B.100d.txt")
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_vectors = KeyedVectors.load_word2vec_format(tmp_file)

In [9]:
from gensim.utils import simple_preprocess
import pandas as pd

vocabulary.staticList = [' '.join(sample) for sample in mid_preprocessed.select("filtered_words").rdd.flatMap(lambda x: x).collect()]
label.staticList = mid_preprocessed.select("label").rdd.flatMap(lambda x: x).collect()

df = pd.DataFrame(zip(vocabulary.staticList, label.staticList), columns = ['sentence_str', 'label'])
# using gensim's built-in tokenizer 
df['gensim_tokenized'] = df['sentence_str'].map(lambda x: simple_preprocess(x.lower(),deacc=True, max_len=100))

In [10]:
# for this to work, your input is going to be a list of tokenized sentences
# since my data was in a dataframe, first I selected the appropriate series
documents = df[df['label'] == 1]['gensim_tokenized']
print(len(documents))
# then made it into a list
sentences = [sentence for sentence in documents]

from sklearn.feature_extraction import _stop_words
stopwords = list(_stop_words.ENGLISH_STOP_WORDS)

cleaned_sentences = []
for sentence in sentences:
    cleaned = [word.lower() for word in sentence]
    cleaned = [word for word in cleaned if word not in stopwords]
    cleaned_sentences.append(cleaned)

99756


In [11]:
cleaned_sentences[:10]

[['threatmeter',
  'hacked',
  'emails',
  'san',
  'francisco',
  'muni',
  'rail',
  'hacker',
  'reveals',
  'clues',
  'identity',
  'tactics',
  'kr',
  'fin'],
 ['android', 'malware', 'targeting', 'pcs', 'uncovered', 'fbxtynm'],
 ['adobe', 'fixes', 'code', 'execution', 'bugs', 'flash', 'mokgxr'],
 ['scienceporn', 'vacuum', 'guess'],
 ['riskware', 'hmoyfzb', 'fbjaz'],
 ['rt',
  'exploithunt',
  'wzemae',
  'data',
  'protection',
  'fusion',
  'itsec',
  'databreach',
  'news',
  'wzenei'],
 ['webroot',
  'spamvertised',
  'lost',
  'message',
  'facebook',
  'campaign',
  'leads',
  'pharmaceutical',
  'scams',
  'xunah'],
 ['cybersecurity', 'hacker', 'privacy', 'big', 'data', 'rmge'],
 ['postsnag', 'gingcredsfromlockedmachines'],
 ['ddos', 'mvrzcc']]

In [12]:
from gensim.models.word2vec import Word2Vec

# build a word2vec model on your dataset
base_model = Word2Vec(size=100, max_final_vocab =20000)
base_model.build_vocab(cleaned_sentences) # update를 통해 나중에 업데이트를 하면될듯.
total_examples = base_model.corpus_count

In [13]:
total_examples

99756

In [14]:
len(list(glove_vectors.vocab.keys())) # pretrained된 glove.6B.100d에 저장된 단어 4만개

400000

In [15]:
# add GloVe's vocabulary & weights
base_model.build_vocab([list(glove_vectors.vocab.keys())], update=True)

# train on your data
base_model.train(cleaned_sentences, total_examples=total_examples, epochs=20) # base_model.epochs
base_model_wv = base_model.wv

In [16]:
glove_vectors.vocab

{'the': <gensim.models.keyedvectors.Vocab at 0x7f7fd80a7f60>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7f7fdb755be0>,
 '.': <gensim.models.keyedvectors.Vocab at 0x7f7fd80a7f98>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7f810e6345f8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7f7fd80b1048>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7f7fd80a7668>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7f7fd80b10b8>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7f7fd80a7710>,
 '"': <gensim.models.keyedvectors.Vocab at 0x7f7fd80b10f0>,
 "'s": <gensim.models.keyedvectors.Vocab at 0x7f7fd80a7748>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7f7fd80b1160>,
 '-': <gensim.models.keyedvectors.Vocab at 0x7f7fd80a77b8>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7f7fd80b1198>,
 'on': <gensim.models.keyedvectors.Vocab at 0x7f7fd80a7828>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7f7fd80b1208>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7f7fd80a7898>,
 'said': <gensim.models

In [17]:
base_model_wv.vocab

{'threatmeter': <gensim.models.keyedvectors.Vocab at 0x7f810c09f5f8>,
 'hacked': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0080>,
 'emails': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0208>,
 'san': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0390>,
 'francisco': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0198>,
 'muni': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e01d0>,
 'rail': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0438>,
 'hacker': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0240>,
 'reveals': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e02b0>,
 'clues': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0048>,
 'identity': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0278>,
 'tactics': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0160>,
 'kr': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e02e8>,
 'fin': <gensim.models.keyedvectors.Vocab at 0x7f7fc42e0588>,
 'android': <gensim.models.keyedvectors.Vocab at 0x7f7faca12160>,
 'malware': <gensim.models.

In [18]:
base_model_wv.save_word2vec_format('/root/spark/w2v_for_nn.bin')
base_model_wv.save('/root/spark/w2v_for_nn.wordvectors')

In [19]:
## Textvectorization

In [149]:
class voc:
    staticList = list() # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class word_index:
    staticDict = dict() # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

In [150]:
class x_train:
    data = None

In [151]:
#계속해서 선언해주기
class a:
    length = len(voc.staticList) # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class b:
    length = len(word_index.staticDict) # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

# class c:
#     length = len(vocabulary.staticList)

In [152]:
from pyspark.ml import Transformer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from tensorflow.keras.preprocessing import sequence

class TextToSequence(Transformer):
    vectorizer = TextVectorization(output_sequence_length=200, vocabulary=list(base_model.wv.vocab.keys()))#max_tokens=20000, 
    tokenizer = Tokenizer(num_words=15308)
    
    def __init__(self):
        super(TextToSequence, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = [' '.join(sample) for sample in df.select("filtered_words").rdd.flatMap(lambda x: x).collect()]
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        
        # train_x = [' '.join(sample) for sample in df.filter(col('label')==1).select("filtered_words").rdd.flatMap(lambda x: x).collect()]
        
        self.tokenizer.fit_on_texts(cleaned_sentences)
        train_sequences = self.tokenizer.texts_to_sequences(samples)
        x_train.data = sequence.pad_sequences(train_sequences, maxlen=200, padding='post')
        
        words = df.select("filtered_words").rdd.flatMap(lambda x: x).collect()
        # 200의 길이로 상위 2만개의 토큰만 반영
#         text_ds = tf.data.Dataset.from_tensor_slices(samples).batch(128)
#         self.vectorizer.adapt(text_ds)
        
        temp = voc.staticList
        print(len(temp))
        # 엄데이트 하는 식으로 구현 동작 바꾸기!
        voc.staticList = self.vectorizer.get_vocabulary() #list(set(voc.staticList + self.vectorizer.get_vocabulary()))
        
        aa = self.vectorizer.get_vocabulary()# list(set(self.vectorizer.get_vocabulary())-set(temp))
        print(len(aa),"!")
        
        post_length = len(voc.staticList)
        word_index.staticDict.update(dict(zip(aa, range(a.length, a.length+len(aa)))))
        
        #x_train.data = self.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_train.data.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf

In [153]:
# # get_tokens 함수과 같은 용도
# stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
# # get_lemma 함수와 같은 용도
# stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
stage_3 = TextToSequence()
stage_4 = StringIndexer(inputCol='label', outputCol='label_index')

transformer_pipeline = Pipeline(stages=[stage_3, stage_4])
preprocessing = transformer_pipeline.fit(mid_preprocessed)
preprocessed = preprocessing.transform(mid_preprocessed)

0
15310 !
15310
15310 !


In [126]:
print(x_train.data[99754:99760])

[[14346  4362   185 ...     0     0     0]
 [    4  3468  2107 ...     0     0     0]
 [  263  2050     0 ...     0     0     0]
 [  432  1131  1656 ...     0     0     0]
 [ 2316 10665   207 ...     0     0     0]
 [  263 11823   862 ...     0     0     0]]


In [89]:
print(x_train.data)

[[   30    65   530 ...     0     0     0]
 [   90    21    12 ...     0     0     0]
 [  159   576  1593 ...     0     0     0]
 ...
 [  221    49   149 ...     0     0     0]
 [14346  4362   185 ...     0     0     0]
 [    4  3468  2107 ...     0     0     0]]


In [26]:
' '.join(mid_preprocessed.select("filtered_words").rdd.flatMap(lambda x: x).collect()[1])

'first android malware targeting pcs uncovered fbxtynm'

In [27]:
mid_preprocessed.select("text").rdd.flatMap(lambda x: x).collect()[1]

'first android malware targeting pcs uncovered  fbxtynm '

In [90]:
TextToSequence.tokenizer.word_index

{'security': 1,
 'cybersecurity': 2,
 'hacker': 3,
 'cve': 4,
 'vulnerability': 5,
 'data': 6,
 'new': 7,
 'hacking': 8,
 'securityaffairs': 9,
 'rt': 10,
 'bitcoin': 11,
 'malware': 12,
 'cyber': 13,
 'infosec': 14,
 'windows': 15,
 'us': 16,
 'remote': 17,
 'hackers': 18,
 'news': 19,
 'google': 20,
 'android': 21,
 'via': 22,
 'web': 23,
 'microsoft': 24,
 'quote': 25,
 'code': 26,
 'exploit': 27,
 'alert': 28,
 'wordpress': 29,
 'threatmeter': 30,
 'attack': 31,
 'vulnerabilities': 32,
 'information': 33,
 'internet': 34,
 'service': 35,
 'apple': 36,
 'attacks': 37,
 'ransomware': 38,
 'day': 39,
 'network': 40,
 'breach': 41,
 'linux': 42,
 'exploits': 43,
 'injection': 44,
 'one': 45,
 'update': 46,
 'users': 47,
 'x': 48,
 'cloud': 49,
 'online': 50,
 'guide': 51,
 'get': 52,
 'hack': 53,
 'software': 54,
 'mobile': 55,
 'using': 56,
 'world': 57,
 'sql': 58,
 'cybercrime': 59,
 'use': 60,
 'facebook': 61,
 'server': 62,
 'execution': 63,
 'multiple': 64,
 'hacked': 65,
 'allow

In [91]:
len(voc.staticList)

15310

In [92]:
len(word_index.staticDict)

15310

In [93]:
word_index.staticDict

{'': 0,
 '[UNK]': 1,
 'threatmeter': 2,
 'hacked': 3,
 'emails': 4,
 'san': 5,
 'francisco': 6,
 'muni': 7,
 'rail': 8,
 'hacker': 9,
 'reveals': 10,
 'clues': 11,
 'identity': 12,
 'tactics': 13,
 'kr': 14,
 'fin': 15,
 'android': 16,
 'malware': 17,
 'targeting': 18,
 'pcs': 19,
 'uncovered': 20,
 'adobe': 21,
 'fixes': 22,
 'code': 23,
 'execution': 24,
 'bugs': 25,
 'flash': 26,
 'vacuum': 27,
 'guess': 28,
 'rt': 29,
 'exploithunt': 30,
 'data': 31,
 'protection': 32,
 'fusion': 33,
 'itsec': 34,
 'databreach': 35,
 'news': 36,
 'webroot': 37,
 'lost': 38,
 'message': 39,
 'facebook': 40,
 'campaign': 41,
 'leads': 42,
 'pharmaceutical': 43,
 'scams': 44,
 'cybersecurity': 45,
 'privacy': 46,
 'big': 47,
 'ddos': 48,
 'actions': 49,
 'runtime': 50,
 'application': 51,
 'watch': 52,
 'algorithm': 53,
 'watching': 54,
 'product': 55,
 'model': 56,
 'forensic': 57,
 'ultradock': 58,
 'ty': 59,
 'bitcoin': 60,
 'securitytube': 61,
 'video': 62,
 'beef': 63,
 'mashup': 64,
 'know': 65,

In [94]:
len(vocabulary.staticList)

199512

In [95]:
len(label.staticList)

199512

In [96]:
list(base_model.wv.vocab.keys())

['threatmeter',
 'hacked',
 'emails',
 'san',
 'francisco',
 'muni',
 'rail',
 'hacker',
 'reveals',
 'clues',
 'identity',
 'tactics',
 'kr',
 'fin',
 'android',
 'malware',
 'targeting',
 'pcs',
 'uncovered',
 'adobe',
 'fixes',
 'code',
 'execution',
 'bugs',
 'flash',
 'vacuum',
 'guess',
 'rt',
 'exploithunt',
 'data',
 'protection',
 'fusion',
 'itsec',
 'databreach',
 'news',
 'webroot',
 'lost',
 'message',
 'facebook',
 'campaign',
 'leads',
 'pharmaceutical',
 'scams',
 'cybersecurity',
 'privacy',
 'big',
 'ddos',
 'actions',
 'runtime',
 'application',
 'watch',
 'algorithm',
 'watching',
 'product',
 'model',
 'forensic',
 'ultradock',
 'ty',
 'bitcoin',
 'securitytube',
 'video',
 'beef',
 'mashup',
 'know',
 'moment',
 'words',
 'impact',
 'life',
 'thezigziglar',
 'success',
 'leadership',
 'cvechrome',
 'google',
 'chrome',
 'allows',
 'remote',
 'atta',
 'cve',
 'schneider',
 'electric',
 'umotion',
 'builder',
 'software',
 'versions',
 'prior',
 'cross',
 'site',
 '

In [98]:
TextToSequence.vectorizer.get_vocabulary()

['',
 '[UNK]',
 'threatmeter',
 'hacked',
 'emails',
 'san',
 'francisco',
 'muni',
 'rail',
 'hacker',
 'reveals',
 'clues',
 'identity',
 'tactics',
 'kr',
 'fin',
 'android',
 'malware',
 'targeting',
 'pcs',
 'uncovered',
 'adobe',
 'fixes',
 'code',
 'execution',
 'bugs',
 'flash',
 'vacuum',
 'guess',
 'rt',
 'exploithunt',
 'data',
 'protection',
 'fusion',
 'itsec',
 'databreach',
 'news',
 'webroot',
 'lost',
 'message',
 'facebook',
 'campaign',
 'leads',
 'pharmaceutical',
 'scams',
 'cybersecurity',
 'privacy',
 'big',
 'ddos',
 'actions',
 'runtime',
 'application',
 'watch',
 'algorithm',
 'watching',
 'product',
 'model',
 'forensic',
 'ultradock',
 'ty',
 'bitcoin',
 'securitytube',
 'video',
 'beef',
 'mashup',
 'know',
 'moment',
 'words',
 'impact',
 'life',
 'thezigziglar',
 'success',
 'leadership',
 'cvechrome',
 'google',
 'chrome',
 'allows',
 'remote',
 'atta',
 'cve',
 'schneider',
 'electric',
 'umotion',
 'builder',
 'software',
 'versions',
 'prior',
 'cros

In [35]:
base_model_wv.most_similar('hacker')

[('bitcoin', 0.798552930355072),
 ('cyberwarbooks', 0.6716892719268799),
 ('cybersecurity', 0.5457794666290283),
 ('reviews', 0.5087110996246338),
 ('handbook', 0.49216559529304504),
 ('tobemcomcyberwarcyber', 0.4836355149745941),
 ('tobemcomcyberwarhacki', 0.47644609212875366),
 ('breakin', 0.47043347358703613),
 ('cyberlaw', 0.46818482875823975),
 ('danger', 0.4664571285247803)]

In [36]:
from gensim.models import KeyedVectors
base_model_wv = KeyedVectors.load('/root/spark/w2v_for_nn.wordvectors')

In [37]:
len(glove_vectors.vocab.keys())

400000

In [38]:
len(base_model.wv.vocab.keys())

15308

In [39]:
# base_model의 vocab 사이즈 확인
base_model_wv.vectors.shape

(15308, 100)

In [183]:
glove_vectors

In [184]:
base_model_wv

In [186]:
len(tokenizer.word_index)

152794

In [189]:
MAX_NB_WORDS = 15308

In [192]:
min([MAX_NB_WORDS, 200])

Py4JError: An error occurred while calling z:org.apache.spark.sql.functions.min. Trace:
py4j.Py4JException: Method min([class java.util.ArrayList]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [204]:
# from keras.layers import Embedding

nb_words = MAX_NB_WORDS
word_index = tokenizer.word_index
i = 0

embedding_matrix = np.zeros((nb_words, 100))
for word, i in word_index.items():
    if i == 15308:
        break
        
    if word in base_model_wv.vocab:
        embedding_matrix[i] = base_model_wv.word_vec(word)
    i = i + 1
    
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

embedding_layer = Embedding(embedding_matrix.shape[0], # or len(word_index) + 1
                            embedding_matrix.shape[1], # or EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=200,
                            trainable=False)

Null word embeddings: 1


In [208]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation


model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(300, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(150,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 200, 100)          1530800   
_________________________________________________________________
dropout_9 (Dropout)          (None, 200, 100)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 99, 300)           90300     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 49, 150)           135150    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 24, 75)            33825     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1800)             

In [209]:
model.fit(x_train, y_train_seq, validation_split=0.2, epochs=10, batch_size=256)
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
624/624 [==============================] - 167s 264ms/step - loss: 0.2571 - acc: 0.8864 - val_loss: 0.2230 - val_acc: 0.9143
Epoch 2/10
624/624 [==============================] - 231s 370ms/step - loss: 0.1873 - acc: 0.9222 - val_loss: 0.3110 - val_acc: 0.8893
Epoch 3/10
 97/624 [===>..........................] - ETA: 3:06 - loss: 0.1672 - acc: 0.9321

KeyboardInterrupt: 

In [177]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
import keras
#For building the Neural Network layer by layer
from keras.models import Sequential
#To randomly initialize the weights to small numbers close to 0(But not 0)
from keras.layers import Dense

embedding_layer = base_model_wv.get_keras_embedding(train_embeddings=False)

In [160]:
base_model_wv.vocab['threatmeter'].index

27

In [133]:
base_model_wv.index2word[:4]

['security', 'cybersecurity', 'hacker', 'cve']

In [140]:
list(TextToSequence.tokenizer.word_index.keys())[:4]

['security', 'cybersecurity', 'hacker', 'cve']

In [147]:
base_model_wv.index2word[:150]

['security',
 'cybersecurity',
 'hacker',
 'cve',
 'vulnerability',
 'data',
 'new',
 'hacking',
 'securityaffairs',
 'rt',
 'bitcoin',
 'malware',
 'cyber',
 'infosec',
 'windows',
 'remote',
 'hackers',
 'news',
 'google',
 'android',
 'web',
 'microsoft',
 'quote',
 'code',
 'exploit',
 'alert',
 'wordpress',
 'threatmeter',
 'attack',
 'vulnerabilities',
 'information',
 'internet',
 'service',
 'apple',
 'attacks',
 'ransomware',
 'day',
 'network',
 'breach',
 'linux',
 'exploits',
 'injection',
 'update',
 'users',
 'cloud',
 'online',
 'guide',
 'hack',
 'software',
 'mobile',
 'using',
 'world',
 'sql',
 'cybercrime',
 'use',
 'facebook',
 'server',
 'execution',
 'multiple',
 'hacked',
 'allows',
 'site',
 'file',
 'business',
 'privacy',
 'dont',
 'vuln',
 'threat',
 'scripting',
 'leadership',
 'cisco',
 'people',
 'denial',
 'big',
 'version',
 'says',
 'like',
 'time',
 'computer',
 'report',
 'ios',
 'book',
 'know',
 'post',
 'analysis',
 'access',
 'overflow',
 'local'

In [154]:
list(TextToSequence.tokenizer.word_index.keys())[:150]

['security',
 'cybersecurity',
 'hacker',
 'cve',
 'vulnerability',
 'data',
 'new',
 'hacking',
 'securityaffairs',
 'rt',
 'bitcoin',
 'malware',
 'cyber',
 'infosec',
 'windows',
 'remote',
 'hackers',
 'news',
 'google',
 'android',
 'web',
 'microsoft',
 'quote',
 'code',
 'exploit',
 'alert',
 'wordpress',
 'threatmeter',
 'attack',
 'vulnerabilities',
 'information',
 'internet',
 'service',
 'apple',
 'attacks',
 'ransomware',
 'day',
 'network',
 'breach',
 'linux',
 'exploits',
 'injection',
 'update',
 'users',
 'cloud',
 'online',
 'guide',
 'hack',
 'software',
 'mobile',
 'using',
 'world',
 'sql',
 'cybercrime',
 'use',
 'facebook',
 'server',
 'execution',
 'multiple',
 'hacked',
 'allows',
 'site',
 'file',
 'business',
 'privacy',
 'dont',
 'vuln',
 'threat',
 'scripting',
 'leadership',
 'cisco',
 'people',
 'denial',
 'big',
 'version',
 'says',
 'like',
 'time',
 'computer',
 'report',
 'ios',
 'book',
 'know',
 'post',
 'analysis',
 'access',
 'overflow',
 'local'

In [164]:
samples = [' '.join(sample) for sample in mid_preprocessed.select("filtered_words").rdd.flatMap(lambda x: x).collect()]

In [165]:
samples

['threatmeter hacked emails san francisco muni rail system hacker reveals clues identity tactics kr fin',
 'first android malware targeting pcs uncovered fbxtynm',
 'adobe fixes six code execution bugs flash mokgxr',
 'scienceporn vacuum guess',
 'riskware hmoyfzb fbjaz',
 'rt exploithunt wzemae data protection fusion itsec databreach news wzenei',
 'webroot spamvertised lost message facebook campaign leads pharmaceutical scams xunah',
 'cybersecurity hacker privacy big data rmge',
 'postsnag gingcredsfromlockedmachines',
 'ddos mvrzcc',
 'top actions runtime application protection fbodttge',
 'watch aqvqvy',
 'threatmeter facebook algorithm watching loetua',
 'product model v product name forensic ultradock v product ty rltxbn cybersecurity bitcoin',
 'rt securitytube video ettercap beef mashup',
 'never know moment sincere words impact life thezigziglar success leadership',
 'cvechrome google chrome allows remote atta',
 'cve schneider electric umotion builder software versions prior

In [166]:
tokenizer = Tokenizer(num_words=15308)

tokenizer.fit_on_texts(cleaned_sentences)
train_sequences = tokenizer.texts_to_sequences(samples)
x_train = sequence.pad_sequences(train_sequences, maxlen=200, padding='post')

In [167]:
x_train

array([[  28,   60,  475, ...,    0,    0,    0],
       [  20,   12,  531, ...,    0,    0,    0],
       [ 147,  516,   24, ...,    0,    0,    0],
       ...,
       [4042,  510,  201, ...,    0,    0,    0],
       [1295,  543,  272, ...,    0,    0,    0],
       [ 388, 6710, 3316, ...,    0,    0,    0]], dtype=int32)

In [170]:
print(list(tokenizer.word_index.keys())[27:29], list(tokenizer.word_index.keys())[59:61])

['threatmeter', 'attack'] ['hacked', 'allows']


In [176]:
base_model_wv.index2word[27]

'threatmeter'

In [174]:
base_model_wv.vocab['threatmeter'].index

27

In [172]:
tokenizer.sequences_to_texts(x_train[:10])

['threatmeter hacked emails san francisco muni rail hacker reveals clues identity tactics kr fin',
 'android malware targeting pcs uncovered',
 'adobe fixes code execution bugs flash',
 'vacuum guess',
 '',
 'rt exploithunt data protection fusion itsec databreach news',
 'webroot lost message facebook campaign leads pharmaceutical scams',
 'cybersecurity hacker privacy big data',
 '',
 'ddos']

In [ ]:
base_model_wv.index2word[:150]

In [104]:
embedding_layer

In [155]:
from tensorflow.keras.utils import to_categorical
y_train_seq = to_categorical(label.staticList)

In [178]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(base_model_wv.get_keras_embedding(train_embeddings=False))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
display(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         1530800   
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_10 (Dense)             (None, 25)                1275      
_________________________________________________________________
dropout_5 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 52        
Total params: 1,562,327
Trainable params: 31,527
Non-trainable params: 1,530,800
_________________________________________________________________


None

In [179]:
x_train.data.shape

(199512, 200)

In [181]:
x_train

array([[  28,   60,  475, ...,    0,    0,    0],
       [  20,   12,  531, ...,    0,    0,    0],
       [ 147,  516,   24, ...,    0,    0,    0],
       ...,
       [4042,  510,  201, ...,    0,    0,    0],
       [1295,  543,  272, ...,    0,    0,    0],
       [ 388, 6710, 3316, ...,    0,    0,    0]], dtype=int32)

In [182]:
custom_embeddings_LSTM_history = model.fit(x_train, y_train_seq, epochs=50, batch_size=256, validation_split=0.2)

Epoch 1/50
624/624 [==============================] - 323s 511ms/step - loss: 0.6642 - accuracy: 0.6243 - val_loss: 0.9592 - val_accuracy: 0.0000e+00
Epoch 2/50
624/624 [==============================] - 323s 518ms/step - loss: 0.6618 - accuracy: 0.6250 - val_loss: 0.9777 - val_accuracy: 0.0000e+00
Epoch 3/50
624/624 [==============================] - 322s 517ms/step - loss: 0.6616 - accuracy: 0.6250 - val_loss: 0.9734 - val_accuracy: 0.0000e+00
Epoch 4/50
624/624 [==============================] - 331s 530ms/step - loss: 0.6616 - accuracy: 0.6250 - val_loss: 0.9798 - val_accuracy: 0.0000e+00
Epoch 5/50
 51/624 [=>............................] - ETA: 4:35 - loss: 0.6635 - accuracy: 0.6212

KeyboardInterrupt: 

In [165]:
base_model_wv.vocab

{'threatmeter': <gensim.models.keyedvectors.Vocab at 0x7fa9c45a6208>,
 'hacked': <gensim.models.keyedvectors.Vocab at 0x7faacde4a8d0>,
 'emails': <gensim.models.keyedvectors.Vocab at 0x7fa9c4c98c18>,
 'san': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686048>,
 'francisco': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686080>,
 'muni': <gensim.models.keyedvectors.Vocab at 0x7fa9c46860b8>,
 'rail': <gensim.models.keyedvectors.Vocab at 0x7fa9c46860f0>,
 'hacker': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686128>,
 'reveals': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686160>,
 'clues': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686198>,
 'identity': <gensim.models.keyedvectors.Vocab at 0x7fa9c46861d0>,
 'tactics': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686208>,
 'kr': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686240>,
 'fin': <gensim.models.keyedvectors.Vocab at 0x7fa9c4686278>,
 'android': <gensim.models.keyedvectors.Vocab at 0x7fa9c46862b0>,
 'malware': <gensim.models.

In [161]:
base_model_wv.get_keras_embedding()

In [ ]:
tokenizer, x_train_seqx_train_seq, x_test_seq, y_train_seq, y_test_seq, weights_dict = nn_setup(x,y, max_length=450, tokenizer_name='custom_w2v_tokenizer.pkl')


In [73]:
# pretrained.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"])

from tensorflow.keras import optimizers, regularizers

optimizer_conf = optimizers.Adam(lr=0.001) #0.0001 RmsRrop
opt_conf = optimizers.serialize(optimizer_conf)

In [74]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(False) # one-hot encoding 여부
estimator.set_nb_classes(1)
estimator.set_num_workers(3)
estimator.set_epochs(100)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_0691d43e4312

In [75]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_with_vectors = preprocessed.select(
    preprocessed["text"],
    preprocessed["label_index"],
    list_to_vector_udf(preprocessed["feature"]).alias("feature")
)

In [76]:
df_with_vectors.show(3)

+--------------------+-----------+--------------------+
|                text|label_index|             feature|
+--------------------+-----------+--------------------+
|threatmeter hacke...|        1.0|[104.0,221.0,1198...|
|first android mal...|        1.0|[58.0,65.0,31.0,1...|
|adobe fixes six c...|        1.0|[435.0,1305.0,201...|
+--------------------+-----------+--------------------+
only showing top 3 rows



In [55]:
import time

dl_pipeline = Pipeline(stages=[estimator])
t1 = time.time()
my_dl = dl_pipeline.fit(df_with_vectors)
t2 = time.time() - t1
print(t2)

>>> Fit model


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 4 times, most recent failure: Lost task 0.3 in stage 24.0 (TID 861, kafka1, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/root/spark/python/pyspark/worker.py", line 377, in main
    process()
  File "/root/spark/python/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/root/spark/python/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.6/dist-packages/elephas/worker.py", line 42, in train
    history = self.model.fit(x_train, y_train, **self.train_config)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(

In [66]:
asdf = np.where(x_train.data < 20003, x_train.data-1, x_train.data) # -1 로 재실험 필요
print(asdf)

[[    1     2     3 ...    -1    -1    -1]
 [    0    15    16 ...    -1    -1    -1]
 [   20    21     0 ...    -1    -1    -1]
 ...
 [ 4931   295     0 ...    -1    -1    -1]
 [  122     0  1501 ...    -1    -1    -1]
 [ 2012     0 11135 ...    -1    -1    -1]]


In [67]:
asdfg = np.where(asdf == -1, 0, asdf)
print(asdfg)

[[    1     2     3 ...     0     0     0]
 [    0    15    16 ...     0     0     0]
 [   20    21     0 ...     0     0     0]
 ...
 [ 4931   295     0 ...     0     0     0]
 [  122     0  1501 ...     0     0     0]
 [ 2012     0 11135 ...     0     0     0]]


In [64]:
asdfgh = np.where(asdf == -2, 0, asdfg)
print(asdfgh)

[[    0     1     2 ...     0     0     0]
 [    0    14    15 ...     0     0     0]
 [   19    20     0 ...     0     0     0]
 ...
 [ 4930   294     0 ...     0     0     0]
 [  121     0  1500 ...     0     0     0]
 [ 2011     0 11134 ...     0     0     0]]


In [139]:
def nn_setup(x, y, max_length=450, tokenizer_name='nn_model.pkl'):  
    x_train, x_test, y_train, y_test = train_test_split(x,y)

    tokenizer = text.Tokenizer(num_words=28331)

    tokenizer.fit_on_texts(x_train)
    train_sequences = tokenizer.texts_to_sequences(x_train)
    test_sequences = tokenizer.texts_to_sequences(x_test)

    tokenizer_pkl = open(tokenizer_name, 'wb')
    pickle.dump(tokenizer, tokenizer_pkl)
    files.download(tokenizer_name)
    tokenizer_pkl.close()

    x_train_seq = sequence.pad_sequences(train_sequences, maxlen=max_length)
    x_test_seq = sequence.pad_sequences(test_sequences, maxlen=max_length)

    y_train_seq = to_categorical(y_train)
    y_test_seq = to_categorical(y_test)

    weights= compute_class_weight('balanced', np.unique(y_train), y_train)
    weights_dict = dict(zip( np.unique(y_train),weights))
    return tokenizer, x_train_seq, x_test_seq, y_train_seq, y_test_seq, weights_dict

In [37]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import pickle

def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ',quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

glove_path = "/root/spark/glove.6B.100d.txt"
pre_glove = glove2dict(glove_path)

In [38]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd

TicketData = pd.DataFrame(zip(vocabulary.staticList, label.staticList), columns = ['sentence_str', 'label'])
print(len(TicketData))
TicketData = TicketData[TicketData['label']==1]
print(len(TicketData))

corpus = TicketData['text'].values # == brown_data

cv = CountVectorizer(ngram_range=(1,1), vocabulary=voc.staticList[1:])
X = cv.fit_transform(corpus)

199512
99756


In [39]:
X.shape

(99756, 19999)

In [40]:
CountVectorizedData=pd.DataFrame(X.toarray(), columns=voc.staticList[1:]) # label이 1인것만 고려

In [41]:
CountVectorizedData['Priority']=TicketData['label']
print(CountVectorizedData.shape)
CountVectorizedData.head()

(99756, 20000)


,prog,mad,mobile,nite,filename,busdriver,circle,fingerprints,cookies,immigration,...,vnc,briefly,sempre,statistical,violated,escalate,fraud,autistic,distributing,Priority
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [42]:
GloveWordVectors = {}
glove_path = "/root/spark/glove.6B.100d.txt"

# Reading Glove Data
with open(glove_path, 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], "float")
        GloveWordVectors[word] = vector

In [43]:
# Creating the list of words which are present in the Document term matrix
WordsVocab=CountVectorizedData.columns[:-1]
 
# Printing sample words
WordsVocab[0:10]

Index(['prog', 'mad', 'mobile', 'nite', 'filename', 'busdriver', 'circle',
       'fingerprints', 'cookies', 'immigration'],
      dtype='object')

In [44]:
len(WordsVocab)

19999

In [45]:
# Converting the text to numeric data
X = cv.transform(TicketData['text'])
CountVecData=pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

In [36]:
CountVecData.iloc[0,:]

                0
prog            0
mad             0
mobile          0
nite            0
               ..
violated        0
escalate        0
fraud           0
autistic        0
distributing    0
Name: 0, Length: 20000, dtype: int64

In [ ]:
# Creating empty dataframe to hold sentences
W2Vec_Data=pd.DataFrame()

# Looping through each row for the data
for i in range(CountVecData.shape[0]):
    
    # initiating a sentence with all zeros
    Sentence = np.zeros(100)

    # Looping thru each word in the sentence and if its present in 
    # the Glove model then storing its vector
    for word in WordsVocab[CountVecData.iloc[i,:]]:
        print(word)
        if word in GloveWordVectors.keys():    
            Sentence=Sentence+GloveWordVectors[word] # vectorwise computation
            
        
    # Appending the sentence to the dataframe
    W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))

In [ ]:
# Checking the new representation for sentences
W2Vec_Data.shape

In [ ]:
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

mittens_model = Mittens(n=100, max_iter=10000, learning_rate=0.001) # small learning rate should be offered 0.0001

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=voc.staticList,
    initial_embedding_dict= pre_glove)

newglove = dict(zip(corp_vocab, new_embeddings))

MemoryError: Unable to allocate 114. GiB for an array with shape (99756, 153018) and data type int64

In [35]:
# Preparing Data frame For machine learning
# Priority column acts as a target variable and other columns as predictors
CountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
CountVectorizedData['Priority']=TicketData['label']
print(CountVectorizedData.shape)
CountVectorizedData.head()

MemoryError: Unable to allocate 367. GiB for an array with shape (199512, 247192) and data type int64

In [80]:
with open('1_voc.pkl','wb') as f: 
    pickle.dump(voc.staticList,f)
    
with open('1_word_index.pkl','wb') as f: 
    pickle.dump(word_index.staticDict,f)
    
with open('1_vocabulary.pkl','wb') as f: 
    pickle.dump(vocabulary.staticList,f)

In [40]:
preprocessed.show(3, truncate=False) # feature는 200짜리 list

+-----------------------------------------------------------------------------------------------------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|text                                                                                                 |label|feature                                                                                                                        

In [41]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_with_vectors = preprocessed.select(
    preprocessed["text"],
    preprocessed["label_index"],
    list_to_vector_udf(preprocessed["feature"]).alias("feature")
)

In [6]:
# load the pre-trained Glove weight
f = open("repo_glove_1.pkl","rb")
newglove = pickle.load(f)

num_tokens = len(voc.staticList) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.staticDict.items():
    embedding_vector = newglove.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

NameError: name 'voc' is not defined

In [43]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.13277 , -0.11802 , -0.063487, ...,  0.10199 ,  1.0979  ,
        -0.17951 ],
       [-0.016233,  0.77261 , -0.052126, ...,  0.067194, -0.2468  ,
        -0.12894 ],
       ...,
       [-0.27371 , -0.40996 ,  0.30181 , ..., -0.12692 ,  1.0237  ,
         0.65182 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]])

In [44]:
word_index.staticDict.items()

dict_items([('', 0), ('mobile', 1), ('filename', 2), ('circle', 3), ('fingerprints', 4), ('cardiology', 5), ('radeon', 6), ('review', 7), ('arte', 8), ('grilled', 9), ('uba', 10), ('lazysunday', 11), ('arti', 12), ('wired', 13), ('jug', 14), ('evolution', 15), ('rex', 16), ('cranberry', 17), ('kale', 18), ('brandon', 19), ('cooper', 20), ('hi', 21), ('voltaire', 22), ('hells', 23), ('ela', 24), ('gu', 25), ('networkcentric', 26), ('uxbridge', 27), ('crop', 28), ('spacex', 29), ('avengers', 30), ('mart', 31), ('errors', 32), ('comptia', 33), ('dread', 34), ('flir', 35), ('cissp', 36), ('siemens', 37), ('mistakes', 38), ('symsasa', 39), ('shepherd', 40), ('whiteclaw', 41), ('general', 42), ('warmed', 43), ('happyprinceday', 44), ('presbyterian', 45), ('williams', 46), ('masks', 47), ('allah', 48), ('herb', 49), ('troubleshooting', 50), ('resuming', 51), ('pull', 52), ('northern', 53), ('chased', 54), ('blackmail', 55), ('plea', 56), ('hybrid', 57), ('ginger', 58), ('assault', 59), ('pond

In [5]:
from tensorflow.keras.layers import Embedding
from tensorflow import keras
nb_classes = 2

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

NameError: name 'num_tokens' is not defined

In [65]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(200,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu", name='f1')(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f2')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f3')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu", name='d1')(x)
preds = layers.Dense(1, activation="sigmoid", name='d2')(x)
pretrained = keras.Model(int_sequences_input, preds, name="pretrained_model")
pretrained.summary()

Model: "pretrained_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 100)          2000200   
_________________________________________________________________
f1 (Conv1D)                  (None, 196, 128)          64128     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
f2 (Conv1D)                  (None, 35, 128)           82048     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 7, 128)            0         
_________________________________________________________________
f3 (Conv1D)                  (None, 3, 128)       

In [66]:
# pretrained.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"])

from tensorflow.keras import optimizers, regularizers

optimizer_conf = optimizers.RMSprop(lr=0.001) #0.0001 RmsRrop
opt_conf = optimizers.serialize(optimizer_conf)

In [67]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(pretrained.to_json())
estimator.set_categorical_labels(False) # one-hot encoding 여부
estimator.set_nb_classes(1)
estimator.set_num_workers(3)
estimator.set_epochs(100)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_23d4948febc0

In [68]:
import time

dl_pipeline = Pipeline(stages=[estimator])
t1 = time.time()
my_dl = dl_pipeline.fit(df_with_vectors)
t2 = time.time() - t1
print(t2)

>>> Fit model
>>> Synchronous training complete.
2511.044001340866


In [69]:
pretrained.save("model_1_ckpt")

INFO:tensorflow:Assets written to: model_1_ckpt/assets


In [70]:
pretrained.save_weights("pretrained_1_ckpt")

In [71]:
kk = list()
kk.append(TextToSequence.vectorizer.get_vocabulary()[0])
kk.append(TextToSequence.vectorizer.get_vocabulary()[1])
kk.append(TextToSequence.vectorizer.get_vocabulary()[2])
kk.append(TextToSequence.vectorizer.get_vocabulary()[3])
kk.append(TextToSequence.vectorizer.get_vocabulary()[4])
kk.append(TextToSequence.vectorizer.get_vocabulary()[5])
kk

['', '[UNK]', 'security', 'cybersecurity', 'job', 'new']

In [9]:
class TextToSequence2(Transformer):
    def __init__(self):
        super(TextToSequence2, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = [' '.join(sample) for sample in df.select("filtered_words").rdd.flatMap(lambda x: x).collect()]
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        
        # 200의 길이로 상위 2만개의 토큰만 반영
        #vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
        
        x_test = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_test.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf
    
# get_tokens 함수과 같은 용도
stage_5 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
# get_lemma 함수와 같은 용도
stage_6 = StopWordsRemover(inputCol= stage_5.getOutputCol(), outputCol= 'filtered_words')
stage_7 = TextToSequence2()
stage_8 = StringIndexer(inputCol='label', outputCol='label_index')
test_pipeline = Pipeline(stages=[stage_5, stage_6, stage_7, stage_8])

In [73]:
preprocessing2 = test_pipeline.fit(second_df)
preprocessed2 = preprocessing2.transform(second_df)

df_test = preprocessed2.select(
    preprocessed2["label_index"],
    list_to_vector_udf(preprocessed2["feature"]).alias("feature")
)

In [74]:
df_test.select('feature').show(3, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|feature                                                                                                                                                                 

In [75]:
pred_test = my_dl.transform(df_test)

In [76]:
pred_test.select('prediction').show(20, truncate=False)

+------------------------+
|prediction              |
+------------------------+
|[1.0]                   |
|[2.2373693609606393E-13]|
|[0.9999977350234985]    |
|[1.0]                   |
|[1.0]                   |
|[0.991092324256897]     |
|[0.99998539686203]      |
|[0.7662193775177002]    |
|[1.0]                   |
|[1.0]                   |
|[0.9931349754333496]    |
|[0.9984681606292725]    |
|[1.0]                   |
|[2.5598984980681926E-31]|
|[1.0]                   |
|[1.0]                   |
|[1.0]                   |
|[1.0]                   |
|[3.578917812774307E-6]  |
|[1.7790301812609034E-16]|
+------------------------+
only showing top 20 rows



In [77]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test2 = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l>=0.5 else 0.0 , DoubleType())
pred_test3 = pred_test2.select(
    pred_test2["label_index"], 
    list_to_vector_udf(pred_test2["prediction2"]).alias("rawPrediction")
)

pred_test3.show(20)

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|        1.0|          1.0|
|        1.0|          0.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          0.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          0.0|
|        1.0|          0.0|
+-----------+-------------+
only showing top 20 rows



In [78]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")
accuracy = evaluator.evaluate(pred_test3)
print(accuracy)

0.8632


In [12]:
m = ['I am hacker in Seoul for samsung']

TextToSequence.vectorizer(np.array([[s] for s in m])).numpy()

array([[   1,    1,    7,    1, 9066,    1, 1065,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [13]:
with open('1_voc.pkl','rb') as f: 
    voc.staticList = pickle.load(f)
    
with open('1_word_index.pkl','rb') as f: 
    word_index.staticDict = pickle.load(f)
    
with open('1_vocabulary.pkl','rb') as f: 
    vocabulary.staticList = pickle.load(f)

In [14]:
#계속해서 선언해주기
class a:
    length = len(voc.staticList) # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class b:
    length = len(word_index.staticDict) # 현재는 계속해서 incremental하게 사이즈가 커지고 있다

class c:
    length = len(vocabulary.staticList)

In [15]:
from pyspark.ml import Transformer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence_revision(Transformer):
    vectorizer2 = TextVectorization(max_tokens=20000, output_sequence_length=200)
    
    def __init__(self):
        super(TextToSequence_revision, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = [' '.join(sample) for sample in df.select("filtered_words").rdd.flatMap(lambda x: x).collect()]
        
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        words = df.select("filtered_words").rdd.flatMap(lambda x: x).collect()
        # 200의 길이로 상위 2만개의 토큰만 반영
        text_ds = tf.data.Dataset.from_tensor_slices(samples).batch(128)
        self.vectorizer2.adapt(text_ds)
        
        temp = voc.staticList
        print(len(temp))
        # 엄데이트 하는 식으로 구현 동작 바꾸기!
        voc.staticList = list(set(voc.staticList + self.vectorizer2.get_vocabulary()))
        
        aa = list(set(self.vectorizer2.get_vocabulary())-set(temp))
        print(len(aa),"!")
        
        post_length = len(voc.staticList)
        vocabulary.staticList = np.concatenate(words).tolist()
        
        word_index.staticDict.update(dict(zip(aa, range(a.length, a.length+len(aa)))))
        
        x_train = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_train.tolist()))
        rdd = sc.parallelize(zip_array, 306) #, numSlices=306
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf

In [16]:
# get_tokens 함수과 같은 용도
stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
# get_lemma 함수와 같은 용도
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')
stage_3 = TextToSequence_revision()
stage_4 = StringIndexer(inputCol='label', outputCol='label_index')

transformer_pipeline = Pipeline(stages=[stage_1, stage_2, stage_3, stage_4])
preprocessing = transformer_pipeline.fit(first_df_df)
preprocessed = preprocessing.transform(first_df_df)

20000
5634 !
25634
0 !


In [109]:
with open('2_voc.pkl','wb') as f: 
    pickle.dump(voc.staticList,f)
    
with open('2_word_index.pkl','wb') as f: 
    pickle.dump(word_index.staticDict,f)
    
with open('2_vocabulary.pkl','wb') as f: 
    pickle.dump(vocabulary.staticList,f)

In [110]:
preprocessed.show(3, truncate=False) # feature는 200짜리 list

+---------------------------------------------------------------------------------------+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|text                                                                                   |label|feature                                                                                                                                                                               

In [17]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_with_vectors = preprocessed.select(
    preprocessed["text"],
    preprocessed["label_index"],
    list_to_vector_udf(preprocessed["feature"]).alias("feature")
)

In [139]:
# load the pre-trained Glove weight
f = open("repo_glove_2.pkl","rb")
newglove = pickle.load(f)

num_tokens = len(voc.staticList) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.staticDict.items():
    embedding_vector = newglove.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 20031 words (5603 misses)


In [113]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.13277 , -0.11802 , -0.063487, ...,  0.10199 ,  1.0979  ,
        -0.17951 ],
       [-0.016233,  0.77261 , -0.052126, ...,  0.067194, -0.2468  ,
        -0.12894 ],
       ...,
       [-0.75831 ,  0.42915 , -0.040031, ...,  0.11268 , -0.088691,
        -0.20963 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]])

In [44]:
word_index.staticDict.items()

dict_items([('', 0), ('mobile', 1), ('filename', 2), ('circle', 3), ('fingerprints', 4), ('cardiology', 5), ('radeon', 6), ('review', 7), ('arte', 8), ('grilled', 9), ('uba', 10), ('lazysunday', 11), ('arti', 12), ('wired', 13), ('jug', 14), ('evolution', 15), ('rex', 16), ('cranberry', 17), ('kale', 18), ('brandon', 19), ('cooper', 20), ('hi', 21), ('voltaire', 22), ('hells', 23), ('ela', 24), ('gu', 25), ('networkcentric', 26), ('uxbridge', 27), ('crop', 28), ('spacex', 29), ('avengers', 30), ('mart', 31), ('errors', 32), ('comptia', 33), ('dread', 34), ('flir', 35), ('cissp', 36), ('siemens', 37), ('mistakes', 38), ('symsasa', 39), ('shepherd', 40), ('whiteclaw', 41), ('general', 42), ('warmed', 43), ('happyprinceday', 44), ('presbyterian', 45), ('williams', 46), ('masks', 47), ('allah', 48), ('herb', 49), ('troubleshooting', 50), ('resuming', 51), ('pull', 52), ('northern', 53), ('chased', 54), ('blackmail', 55), ('plea', 56), ('hybrid', 57), ('ginger', 58), ('assault', 59), ('pond

In [20]:
word_index.staticDict.items()

dict_items([('', 0), ('mobile', 1), ('filename', 2), ('circle', 3), ('fingerprints', 4), ('cardiology', 5), ('radeon', 6), ('review', 7), ('arte', 8), ('grilled', 9), ('uba', 10), ('lazysunday', 11), ('arti', 12), ('wired', 13), ('jug', 14), ('evolution', 15), ('rex', 16), ('cranberry', 17), ('kale', 18), ('brandon', 19), ('cooper', 20), ('hi', 21), ('voltaire', 22), ('hells', 23), ('ela', 24), ('gu', 25), ('networkcentric', 26), ('uxbridge', 27), ('crop', 28), ('spacex', 29), ('avengers', 30), ('mart', 31), ('errors', 32), ('comptia', 33), ('dread', 34), ('flir', 35), ('cissp', 36), ('siemens', 37), ('mistakes', 38), ('symsasa', 39), ('shepherd', 40), ('whiteclaw', 41), ('general', 42), ('warmed', 43), ('happyprinceday', 44), ('presbyterian', 45), ('williams', 46), ('masks', 47), ('allah', 48), ('herb', 49), ('troubleshooting', 50), ('resuming', 51), ('pull', 52), ('northern', 53), ('chased', 54), ('blackmail', 55), ('plea', 56), ('hybrid', 57), ('ginger', 58), ('assault', 59), ('pond

In [140]:
num_tokens

25636

In [148]:
from tensorflow.keras.layers import Embedding
from tensorflow import keras
nb_classes = 2

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    name='embeddingwithnew'
)

In [149]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(200,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu", name='f1')(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f2')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu", name='f3')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu", name='d1')(x)
preds = layers.Dense(1, activation="sigmoid", name='d2')(x)
pretrained_model = keras.Model(int_sequences_input, preds, name="pretrained")

pretrained_model.summary()

Model: "pretrained"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embeddingwithnew (Embedding) (None, 200, 100)          2563600   
_________________________________________________________________
f1 (Conv1D)                  (None, 196, 128)          64128     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 39, 128)           0         
_________________________________________________________________
f2 (Conv1D)                  (None, 35, 128)           82048     
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 7, 128)            0         
_________________________________________________________________
f3 (Conv1D)                  (None, 3, 128)            8

In [169]:
# from tensorflow.keras import layers

# int_sequences_input = keras.Input(shape=(200,), dtype="int64")
# embedded_sequences = embedding_layer(int_sequences_input)
# x = layers.Conv1D(128, 5, activation="relu", name='f1')(embedded_sequences)
# x = layers.MaxPooling1D(5)(x)
# x = layers.Conv1D(128, 5, activation="relu", name='f2')(x)
# x = layers.MaxPooling1D(5)(x)
# x = layers.Conv1D(128, 5, activation="relu", name='f3')(x)
# x = layers.GlobalMaxPooling1D()(x)
# pretrained_model = keras.Model(int_sequences_input, x, name="pretrained")

# model = keras.Sequential([pretrained_model, layers.Dense(128, activation="relu", name='d1'), layers.Dense(2, activation="softmax", name='d2')])
# model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pretrained (Functional)      (None, 128)               2791824   
_________________________________________________________________
d1 (Dense)                   (None, 128)               16512     
_________________________________________________________________
d2 (Dense)                   (None, 2)                 258       
Total params: 2,808,594
Trainable params: 244,994
Non-trainable params: 2,563,600
_________________________________________________________________


In [170]:
# pretrained_model.summary()

Model: "pretrained"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 200)]             0         
_________________________________________________________________
embeddingwithnew (Embedding) (None, 200, 100)          2563600   
_________________________________________________________________
f1 (Conv1D)                  (None, 196, 128)          64128     
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 39, 128)           0         
_________________________________________________________________
f2 (Conv1D)                  (None, 35, 128)           82048     
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 7, 128)            0         
_________________________________________________________________
f3 (Conv1D)                  (None, 3, 128)            8

In [22]:
model2 = tf.keras.models.load_model("model_1_ckpt")
weights_list = model2.get_weights()

In [150]:
for i, layer in enumerate(pretrained_model.layers):
    print(i,'\t',layer.trainable,'\t  :',layer.name)

0 	 True 	  : input_9
1 	 False 	  : embeddingwithnew
2 	 True 	  : f1
3 	 True 	  : max_pooling1d_16
4 	 True 	  : f2
5 	 True 	  : max_pooling1d_17
6 	 True 	  : f3
7 	 True 	  : global_max_pooling1d_8
8 	 True 	  : d1
9 	 True 	  : d2


In [151]:
for i, layer in enumerate(model2.layers):
    print(i,'\t',layer.trainable,'\t  :',layer.name)

0 	 True 	  : input_4
1 	 True 	  : embedding_1
2 	 True 	  : f1
3 	 True 	  : max_pooling1d_6
4 	 True 	  : f2
5 	 True 	  : max_pooling1d_7
6 	 True 	  : f3
7 	 True 	  : global_max_pooling1d_3
8 	 True 	  : d1
9 	 True 	  : d2


In [152]:
for i in range(len(pretrained_model.get_weights())):
    print(pretrained_model.get_weights()[i].shape)

(25636, 100)
(5, 100, 128)
(128,)
(5, 128, 128)
(128,)
(5, 128, 128)
(128,)
(128, 128)
(128,)
(128, 1)
(1,)


In [153]:
for i in range(len(weights_list)):
    print(weights_list[i].shape)

(20002, 100)
(5, 100, 128)
(128,)
(5, 128, 128)
(128,)
(5, 128, 128)
(128,)
(128, 128)
(128,)
(128, 1)
(1,)


In [154]:
for i, layer in enumerate(pretrained_model.layers):
    if layer.trainable_weights:
        print(layer.name, weights_list[i].shape, pretrained_model.layers[i].name, pretrained_model.get_weights()[i].shape)
        layer.set_weights(model2.layers[i].get_weights())

f1 (128,) f1 (128,)
f2 (128,) f2 (128,)
f3 (128,) f3 (128,)
d1 (128,) d1 (128,)
d2 (128, 1) d2 (128, 1)


In [157]:
pretrained_model.layers[1].set_weights([embedding_matrix])
pretrained_model.layers[1].trainable = True
print(pretrained_model.layers[1].name)

embeddingwithnew


In [158]:
# Check that all of the pretrained weights have been loaded.
for a, b in zip(model2.weights, pretrained_model.weights):
    try:
        np.testing.assert_allclose(a.numpy(), b.numpy())
        print("성공!!!!!!!!!!!!!!!")
    except Exception as e:
        print('예외가 발생했습니다.', e)

예외가 발생했습니다. 
Not equal to tolerance rtol=1e-07, atol=0

(shapes (20002, 100), (25636, 100) mismatch)
 x: array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.13277 , -0.11802 , -0.063487, ...,  0.10199 ,  1.0979  ,...
 y: array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.13277 , -0.11802 , -0.063487, ...,  0.10199 ,  1.0979  ,...
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!
성공!!!!!!!!!!!!!!!


In [159]:
# print('model를 동결하기 전 훈련되는 가중치의 수:',
#      len(model.trainable_weights))
print('model를 동결하기 전 훈련되는 가중치의 수:',
     len(pretrained_model.trainable_weights))

# for layer in pretrained_model.layers[:-4]: 
#     layer.trainable = False
    
# print('model를 동결한 후 훈련되는 가중치의 수:',
#      len(model.trainable_weights))
print('model를 동결한 후 훈련되는 가중치의 수:',
     len(pretrained_model.trainable_weights))

model를 동결하기 전 훈련되는 가중치의 수: 11
model를 동결한 후 훈련되는 가중치의 수: 11


In [160]:
for i, layer in enumerate(pretrained_model.layers):
    print(i,'\t',layer.trainable,'\t  :',layer.name)

0 	 True 	  : input_9
1 	 True 	  : embeddingwithnew
2 	 True 	  : f1
3 	 True 	  : max_pooling1d_16
4 	 True 	  : f2
5 	 True 	  : max_pooling1d_17
6 	 True 	  : f3
7 	 True 	  : global_max_pooling1d_8
8 	 True 	  : d1
9 	 True 	  : d2


In [161]:
pretrained_model.compile(loss="binary_crossentropy", optimizer="SGD", metrics=["acc"])

In [162]:
from tensorflow.keras import optimizers, regularizers

optimizer_conf = optimizers.SGD(lr=0.0001) #0.0001
opt_conf = optimizers.serialize(optimizer_conf)

In [163]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator2 = ElephasEstimator()
estimator2.setFeaturesCol("feature")
estimator2.setLabelCol("label_index")
estimator2.set_keras_model_config(pretrained_model.to_json())
estimator2.set_categorical_labels(False) # one-hot encoding 여부
estimator2.set_nb_classes(1)
estimator2.set_num_workers(3)
estimator2.set_epochs(100)
estimator2.set_batch_size(254)
estimator2.set_verbosity(1)
estimator2.set_validation_split(0.20)
estimator2.set_optimizer_config(opt_conf)
estimator2.set_mode("synchronous")
estimator2.set_loss("binary_crossentropy")
estimator2.set_metrics(['acc'])

ElephasEstimator_85d5bd898fac

In [164]:
new_pipeline = Pipeline(stages=[estimator2])

In [165]:
import time
t1 = time.time()
new_dl = new_pipeline.fit(df_with_vectors)
t2 = time.time() - t1
print(t2) #2511

>>> Fit model
>>> Synchronous training complete.
2338.503976583481


In [166]:
pretrained_model.save("model_2_ckpt")
pretrained_model.save_weights("pretrained_2_ckpt")

INFO:tensorflow:Assets written to: model_2_ckpt/assets


In [67]:
class TextToSequence2(Transformer):
    def __init__(self):
        super(TextToSequence2, self).__init__()
        
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = [' '.join(sample) for sample in df.select("filtered_words").rdd.flatMap(lambda x: x).collect()]
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        
        # 200의 길이로 상위 2만개의 토큰만 반영
        #vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
        
        x_test = TextToSequence.vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_test.tolist()))
        rdd = sc.parallelize(zip_array, 306)
        fdf = rdd.toDF(['text','label','feature'])
        
        return fdf
    
# get_tokens 함수과 같은 용도
stage_5 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')
# get_lemma 함수와 같은 용도
stage_6 = StopWordsRemover(inputCol= stage_5.getOutputCol(), outputCol= 'filtered_words')
stage_7 = TextToSequence2()
stage_8 = StringIndexer(inputCol='label', outputCol='label_index')
test_pipeline = Pipeline(stages=[stage_5, stage_6, stage_7, stage_8])

In [68]:
preprocessing2 = test_pipeline.fit(second_df)
preprocessed2 = preprocessing2.transform(second_df)
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df_test = preprocessed2.select(
    preprocessed2["label_index"],
    list_to_vector_udf(preprocessed2["feature"]).alias("feature")
)

In [40]:
df_test.select('feature').show(3, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|feature                                                                                                                                                                 

In [167]:
pred_test_second = new_dl.transform(df_test)

In [168]:
pred_test_second.select('prediction').show(20, truncate=False)

+--------------------+
|prediction          |
+--------------------+
|[0.5191552639007568]|
|[0.5454633235931396]|
|[0.527237057685852] |
|[0.5315566062927246]|
|[0.537665843963623] |
|[0.5287355184555054]|
|[0.5175348520278931]|
|[0.5036687254905701]|
|[0.5285661816596985]|
|[0.5252978205680847]|
|[0.5205573439598083]|
|[0.5162758827209473]|
|[0.5194987654685974]|
|[0.5370163321495056]|
|[0.5237757563591003]|
|[0.5166089534759521]|
|[0.5234166383743286]|
|[0.5579507946968079]|
|[0.5368083119392395]|
|[0.5159834623336792]|
+--------------------+
only showing top 20 rows



In [169]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test2 = pred_test_second.select(
    pred_test_second["label_index"], 
    list_to_double(pred_test_second["prediction"]).alias("prediction2")
)

from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l>=0.5 else 0.0 , DoubleType())
pred_test3 = pred_test2.select(
    pred_test2["label_index"], 
    list_to_vector_udf(pred_test2["prediction2"]).alias("rawPrediction")
)

pred_test3.show(20)

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
|        1.0|          1.0|
+-----------+-------------+
only showing top 20 rows



In [145]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")
accuracy = evaluator.evaluate(pred_test3)
print(accuracy)

0.51005


In [147]:
newglove

{'the': array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
        -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
         0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
        -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
         0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
        -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
         0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
         0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
        -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
        -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
        -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
        -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
        -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
        -1.2526  ,  0.071624,  

In [146]:
m = ['I am hacker in Seoul for samsung']

TextToSequence.vectorizer(np.array([[s] for s in m])).numpy()

array([[   1,    1,    7,    1, 9066,    1, 1065,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [ ]:
array([[   1,    1,    7,    1, 9066,    1, 1065,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]])

In [44]:
# load the pre-trained Glove weight
f = open("repo_glove_1.pkl","rb")
newglove = pickle.load(f)

In [50]:
print(newglove.get('the'))

[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.27062 ]

In [48]:
# load the pre-trained Glove weight
f = open("repo_glove_2.pkl","rb")
newglove2 = pickle.load(f)

In [51]:
print(newglove2.get('the'))

[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.27062 ]